In [1]:
%load_ext sql

In [2]:
%sql sqlite:///myuniversity.db

In [3]:
from sqlalchemy import create_engine, Column, Integer, String, DECIMAL, Date, Time, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

In [4]:
Base = declarative_base()

class Department(Base):
    __tablename__ = 'Department'
    dept_name = Column(String(50), primary_key=True)
    building = Column(String(50))
    budget = Column(DECIMAL(12, 2))

class Course(Base):
    __tablename__ = 'Course'
    course_ID = Column(String(10), primary_key=True)
    title = Column(String(100))
    dept_name = Column(String(50), ForeignKey('Department.dept_name'))
    credits = Column(Integer)
    syllabus = Column(Text)
    department = relationship("Department", back_populates="courses")
    prerequisites = relationship("Prerequisite", foreign_keys="[Prerequisite.course_ID]",
                                 back_populates="course")
    prerequisites_for = relationship("Prerequisite", foreign_keys="[Prerequisite.prereq_ID]",
                                     back_populates="prerequisite")

Department.courses = relationship("Course", order_by=Course.course_ID, back_populates="department")

class Instructor(Base):
    __tablename__ = 'Instructor'
    ID = Column(String(10), primary_key=True)
    name = Column(String(100))
    dept_name = Column(String(50), ForeignKey('Department.dept_name'))
    salary = Column(DECIMAL(12, 2))
    department = relationship("Department", back_populates="instructors")

Department.instructors = relationship("Instructor", order_by=Instructor.ID, back_populates="department")

class Section(Base):
    __tablename__ = 'Section'
    sec_ID = Column(String(10), primary_key=True)
    course_ID = Column(String(10), ForeignKey('Course.course_ID'), primary_key=True)
    semester = Column(String(10), primary_key=True)
    year = Column(Integer, primary_key=True)
    building = Column(String(50))
    room_no = Column(String(10))
    time_slot_ID = Column(String(10), ForeignKey('Time_Slot.time_slot_ID'))

class Time_Slot(Base):
    __tablename__ = 'Time_Slot'
    time_slot_ID = Column(String(10), primary_key=True)
    day = Column(String(10))
    start_hr = Column(Time)
    end_hr = Column(Time)
    start_min = Column(Time)
    end_min = Column(Time)

class Classroom(Base):
    __tablename__ = 'Classroom'
    room_no = Column(String(10), primary_key=True)
    building = Column(String(50), primary_key=True)
    capacity = Column(Integer)

Section.classroom = relationship("Classroom", foreign_keys=[Section.building, Section.room_no], backref="sections")

class Student(Base):
    __tablename__ = 'Student'
    ID = Column(String(10), primary_key=True)
    name = Column(String(100))
    dept_name = Column(String(50), ForeignKey('Department.dept_name'))
    tot_credit = Column(Integer)
    date_of_birth = Column(Date)

Department.students = relationship("Student", order_by=Student.ID, back_populates="department")

class Advisor(Base):
    __tablename__ = 'Advisor'
    s_ID = Column(String(10), ForeignKey('Student.ID'), primary_key=True)
    I_ID = Column(String(10), ForeignKey('Instructor.ID'))

class Takes(Base):
    __tablename__ = 'Takes'
    ID = Column(String(10), ForeignKey('Student.ID'), primary_key=True)
    course_ID = Column(String(10), ForeignKey('Course.course_ID'), primary_key=True)
    sec_ID = Column(String(10), primary_key=True)
    semester = Column(String(10), primary_key=True)
    year = Column(Integer, primary_key=True)
    grade = Column(String(2))

class Teaches(Base):
    __tablename__ = 'Teaches'
    ID = Column(String(10), ForeignKey('Instructor.ID'), primary_key=True)
    course_ID = Column(String(10), ForeignKey('Course.course_ID'), primary_key=True)
    sec_ID = Column(String(10), primary_key=True)
    semester = Column(String(10), primary_key=True)
    year = Column(Integer, primary_key=True)

class Prerequisite(Base):
    __tablename__ = 'Prerequisite'
    course_ID = Column(String(10), ForeignKey('Course.course_ID'), primary_key=True)
    prereq_ID = Column(String(10), ForeignKey('Course.course_ID'), primary_key=True)
    course = relationship("Course", foreign_keys=[course_ID], back_populates="prerequisites")
    prerequisite = relationship("Course", foreign_keys=[prereq_ID], back_populates="prerequisites_for")

class Grading_Components(Base):
    __tablename__ = 'Grading_Components'
    course_ID = Column(String(10), ForeignKey('Course.course_ID'), primary_key=True)
    max_points = Column(Integer)
    weights = Column(DECIMAL(5, 2))
    course = relationship("Course", back_populates="grading_components")

Course.grading_components = relationship("Grading_Components", uselist=False, back_populates="course")

# Set up the database engine
engine = create_engine('sqlite:///:memory:', echo=True)

# Create all tables in the engine
Base.metadata.create_all(engine)

2024-04-27 22:05:01,451 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-27 22:05:01,452 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Department")
2024-04-27 22:05:01,452 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-27 22:05:01,453 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Department")
2024-04-27 22:05:01,453 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-27 22:05:01,454 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Course")
2024-04-27 22:05:01,454 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-27 22:05:01,455 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Course")
2024-04-27 22:05:01,455 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-27 22:05:01,456 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Instructor")
2024-04-27 22:05:01,456 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-27 22:05:01,457 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Instructor")
2024-04-27 22:05:01,457 INFO sqlalchemy.engine.Engine [r

/var/folders/tf/6tszc4p95dn6fjrhhsz6w6380000gn/T/ipykernel_23456/3660679694.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
Session = sessionmaker(bind=engine)
session = Session()

# Sample data for the Department table
departments = [
    Department(dept_name="Computer Science", building="CompSci Building", budget=500000.00),
    Department(dept_name="Mathematics", building="Math Building", budget=300000.00),
    Department(dept_name="Physics", building="Physics Building", budget=250000.00),
    Department(dept_name="History", building="History Building", budget=200000.00),
    Department(dept_name="English", building="English Building", budget=150000.00),
    Department(dept_name="Biology", building="Biology Building", budget=350000.00),
    Department(dept_name="Chemistry", building="Chemistry Building", budget=360000.00),
    Department(dept_name="Economics", building="Economics Building", budget=400000.00),
    Department(dept_name="Law", building="Law Building", budget=600000.00),
    Department(dept_name="Psychology", building="Psychology Building", budget=220000.00)
]
session.add_all(departments)

InvalidRequestError: Mapper 'Mapper[Student(Student)]' has no property 'department'.  If this property was indicated from other mappers or configure events, ensure registry.configure() has been called.